In [ ]:
%reload_ext autoreload
%autoreload 2
import os,sys
sys.path.insert(1, os.path.join(sys.path[0], '..', 'module'))
import wiki

# Get index of articles

* [all indices on Wikipedia](https://en.wikipedia.org/wiki/Portal:Contents/Indices)
* topics not searched
* international trade ("topics"), theory of constraints (small)
* too big: mathematics, neuroscience

In [ ]:
path_base = '/Users/harangju/Developer/data/wiki/dumps/'
name_xml = 'enwiki-20190801-pages-articles-multistream.xml.bz2'
name_index = 'enwiki-20190801-pages-articles-multistream-index.txt.bz2'
path_xml = path_base + name_xml
path_index = path_base + name_index
dump = wiki.Dump(path_xml, path_index)

In [ ]:
# natural & physical sciences
topics = ['anatomy', 'biochemistry', 'cognitive science', 'evolutionary biology',
          'genetics', 'immunology', 'molecular biology']
topics += ['chemistry', 'biophysics', 'energy', 'optics', 
           'earth science', 'geology', 'meteorology']
# philosophy
# topics += []
topics += ['philosophy of language', 'philosophy of law', 
           'philosophy of mind', 'philosophy of science']
# social sciences
topics += ['economics', 'accounting', 'education', 'linguistics', 'law', 'psychology',
           'sociology']
# technology & applied sciences
topics += ['electronics', 'software engineering', 'robotics']

In [ ]:
links = {}
for topic in topics:
    dump.load_page('Index of %s articles' % topic)
    links[topic] = [str(l) for l in dump.article_links]
    print('Topic "' + topic + '" has ' + str(len(links[topic])) + ' articles.')

In [ ]:
# https://en.wikipedia.org/wiki/Lists_of_mathematics_topics
# algebra
math_topics = ['calculus', 'geometry', 'abstract algebra',
               'Boolean algebra', 'commutative algebra',# 'homological algebra',
               'group theory',# 'representation theory', 
               'linear algebra']
# calculus & analysis
# math_topics += ['complex analysis', 'functional analysis',
#                 'integration and measure theory', 'harmonic analysis',
#                 'Fourier analysis', 'multivariable calculus', 'real analysis',
#                 'variational']
# geometry
# math_topics += ['geometry', 'curves', 'triangle', 'circle', 'general topology',
#                 'differential geometry', 'algebraic geometry', 'algebraic topology',
#                 'geometric topology', 'know theory', 'Lie groups']
# number theory
math_topics += [#'algebraic number theory',
                'number theory']
# applied math
math_topics += ['dynamical systems and differential equations']
#                 'partial differential equation']
topics += math_topics

In [ ]:
links = {}
for topic in math_topics:
    dump.load_page(f"List of {topic} topics")
    links[topic] = [str(l) for l in dump.article_links]
    print('Topic "' + topic + '" has ' + str(len(links[topic])) + ' articles.')

In [ ]:
topics += ['physics']

In [ ]:
import string

topic = 'physics'
links[topic] = []
for letter in ['!$@', '0–9'] + list(string.ascii_uppercase):
    dump.load_page('Index of physics articles (%s)' % letter)
    links[topic].extend([str(l) for l in dump.article_links])
print('Topic "' + topic + '" has ' + str(len(links[topic])) + ' articles.')

In [ ]:
topics += ['mathematics']

In [ ]:
topic = 'mathematics'
links[topic] = []
for letter in ['0–9'] + list(string.ascii_uppercase):
    dump.load_page(
        f"Wikipedia:WikiProject Mathematics/List of mathematics articles ({letter})'
    )
    links[topic].extend([str(l) for l in dump.article_links])
print('Topic "' + topic + '" has ' + str(len(links[topic])) + ' articles.')

# Build graphs of topics

In [ ]:
import pickle
import gensim.utils as gu

path_models = '/Users/harangju/Developer/data/wiki/models/'
tfidf = gu.SaveLoad.load(path_models + 'tfidf.model')
dct = pickle.load(open(path_models + 'dict.model','rb'))

## One network per topic

In [ ]:
path = '/Users/harangju/Developer/data/wiki/graphs/dated/'

networks = {}
for topic in topics:
    ls = links[topic]
    print('\nTopic: ' + topic)
    networks[topic] = wiki.Net()
    networks[topic].build_graph(
        name=topic, dump=dump, nodes=ls, model=tfidf, dct=dct
    )
    networks[topic].save_graph(path + topic + '.pickle')
    networks[topic].save_graph(path + topic + '.gexf')
    networks[topic].save_barcodes(path + topic + '.barcode')

## Subnetworks

## Big network

In [ ]:
all_links = list(set([v for l in links.values() for v in l]))
len(all_links)

In [ ]:
path = '/Users/harangju/Developer/data/wiki/graphs/dated/'

big_network = wiki.Net()
big_network.build_graph(
    name='big_network',
    dump=dump, 
    nodes=all_links, 
    model=tfidf, 
    dct=dct,
    compute_core_periphery=False, 
    compute_communities=False, 
    compute_community_cores=False
)
big_network.save_graph(os.path.join(path, 'big_network_physics_math.pickle'))
big_network.save_graph(os.path.join(path, 'big_network_physics_math.gexf'))
# big_network.save_barcodes(os.path.join(path, 'big_network.barcode'))

## Nodes without years

In [ ]:
path = '/Users/harangju/Developer/data/wiki/graphs/dated-noyear/'

if not os.path.isdir(path):
    os.mkdir(path)

networks_noyear = {}
for topic in topics:
    print('\nTopic: ' + topic)
    networks_noyear[topic] = wiki.Net()
    networks_noyear[topic].build_graph(
        name=topic, dump=dump, nodes=links[topic],
        fill_empty_years=False,
        compute_core_periphery=False,
        compute_communities=False,
        compute_community_cores=False
    )
    networks_noyear[topic].save_graph(path + topic + '.pickle')
    networks_noyear[topic].save_graph(path + topic + '.gexf')

In [ ]:
path = '/Users/harangju/Developer/data/wiki/graphs/dated-noyear/'

networks_noyear = {}
for topic in topics:
    print(topic, end=' ')
    networks_noyear[topic] = wiki.Net()
    networks_noyear[topic].load_graph(path + topic + '.pickle')

In [ ]:
import pandas as pd
import networkx as nx
import plotly.express as px

fraction_years = pd.DataFrame(
    [
        [
            topic,
            len([
                y
                for n, y in nx.get_node_attributes(
                    networks_noyear[topic].graph, 'year'
                ).items()
                if y
            ]) / len(networks_noyear[topic].graph.nodes)
        ]
        for topic in topics
    ],
    columns=['topics', 'fraction']
)
fraction_years

In [ ]:
# for topic in topics:
#     print(topic, end='\t')
#     print( 
#         len([
#             y
#             for n, y in nx.get_node_attributes(
#                 networks_noyear[topic].graph, 'year'
#             ).items()
#             if y
#         ]) / len(networks_noyear[topic].graph.nodes)
#     )

In [ ]:
path_fig = '/Users/harangju/Library/Mobile Documents/com~apple~CloudDocs/' +\
    'Documents/research/wikipedia/results'
path_plot = '0 graphs'

fig = px.histogram(fraction_years.fraction)
fig.update_layout(
    width=500, height=360,
    template='plotly_white',
    xaxis={'range': [0, 1]},
    showlegend=False
)
fig.show()
fig.write_image(os.path.join(path_fig, path_plot, 'fraction_years_with_math.pdf'))

# Generate null networks

## Load networks

In [ ]:
path_saved = '/Users/harangju/Developer/data/wiki/graphs/dated/'
networks = {}
for topic in topics:
    print(topic, end=' ')
    networks[topic] = wiki.Net()
    networks[topic].load_graph(path_saved + topic + '.pickle')

## Randomized target & year

Just randomizing year -> you get the same structures, it's just a matter of when you get those structures.
If you randomize year & target, then you're randomizing the structure & how they come about without changing any basic network statistics.

In [ ]:
null_models = ['target', 'year']
num_nulls = 10
nulls = {}
for null_model in null_models:
    print('Null model: ' + null_model)
    path_to_save_null = '/Users/harangju/Developer/data/wiki/graphs/null-'\
                        +null_model+'/'
    nulls[null_model] = {}
    for topic, network in networks.items():
        print('Topic: ' + topic)
        nulls[null_model][topic] = []
        for i in range(num_nulls):
            print('Null: ' + str(i))
            null = network.randomize(null_model)
            null.graph.name = topic+'-null-'+str(i)
            null.save_graph(path_to_save_null + null.graph.name + '.pickle')
            null.save_barcodes(path_to_save_null + null.graph.name + '.barcode')
            nulls[null_model][topic].append(null)

## Jittered years

In [ ]:
num_jitters = 1
max_jitter = 1
null_model = 'jitter'
path_to_save_null = '/Users/harangju/Developer/data/wiki/graphs/null-'+null_model+'/'
if not os.path.isdir(path_to_save_null):
    os.mkdir(path_to_save_null)
jittered = {}

In [ ]:
import copy
import numpy as np
import numpy.random

for topic, network in networks.items():
    print('Topic: ' + topic)
    jittered[topic] = []
    print('Null: ', end='')
    for i in range(num_jitters):
        print(str(i), end=' ')
        null = wiki.Net()
        null.graph = copy.deepcopy(network.graph)
        for node in null.graph.nodes:
            null.graph.nodes[node]['year'] = null.graph.nodes[node]['year'] +\
                np.random.randint(-max_jitter, max_jitter+1)
        null.graph.name = topic+'-null-'+str(i)
        null.save_graph(path_to_save_null + null.graph.name + '.pickle')
        null.save_barcodes(path_to_save_null + null.graph.name + '.barcode')
        jittered[topic].append(null)
    print()

Gephi notes
* node size, fruchterman reingold = [10, 40], force atlas 2 = [4 16]
* text size = [1 1.4]
* preview font size = 5

## Generative networks
* random geometric graph (modularity)
* stochastic block model (modularity)
* caveman graph (modularity, cliques, most clustered & sparse)
* random clustered graph (clustering)

In [ ]:
num_nulls = 10
gen_functions = {
    'rgg': lambda g: nx.random_geometric_graph(
        g.number_of_nodes(), 
    ),
    'sbm': lambda g: nx.stochastic_block_model(
        
    ),
    'cg': lambda g: nx.caveman_graph(
        
    ),
    'rcg': lambda g: nx.random_clustered_graph(
        
    )
}

In [ ]:
num_nulls = 10
gen_nulls = {}
for name, function in gen_functions.items():
    print('Null model: ' + null_model)
    path_to_save_null = '/Users/harangju/Developer/data/wiki/graphs/null-'+\
        null_model+'/'
    nulls[null_model] = {}
    for topic, network in networks.items():
        print('Topic: ' + topic)
        nulls[null_model][topic] = []
        for i in range(num_nulls):
            print('Null: ' + str(i))
            null = network.randomize(null_model)
            null.graph.name = topic+'-null-'+str(i)
            null.save_graph(path_to_save_null + null.graph.name + '.pickle')
            null.save_barcodes(path_to_save_null + null.graph.name + '.barcode')
            nulls[null_model][topic].append(null)